#Connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/[2023-2024] AN2DL/Homework 1

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/[2023-2024] AN2DL/Homework 1


#Import libraries and set parameters

In [ ]:
# Fix randomness and hide warnings
seed = 80

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.14.0


In [ ]:
# Import other libraries
#library for computer vision
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

#Data upload

In [ ]:
data=np.load('public_data.npz', allow_pickle=True)
images_not_normalized = data['data']
labels_strings= data['labels']
label_map = {"healthy": 0, "unhealthy": 1}
labels = np.vectorize(label_map.get)(labels_strings)
print(images_not_normalized.shape)
print(labels.shape)

(5200, 96, 96, 3)
(5200,)


#Normalization and cleaning of data

In [ ]:
#Normalize images
images=[]
for img in images_not_normalized:
  img=(img/255).astype(np.float32)
  images.append(img)

images= np.array(images)
print(img)

[[[0.25490198 0.42745098 0.11764706]
  [0.30588236 0.49019608 0.17254902]
  [0.33333334 0.54901963 0.19215687]
  ...
  [0.25882354 0.48235294 0.17254902]
  [0.23921569 0.4627451  0.15294118]
  [0.20392157 0.42745098 0.11764706]]

 [[0.3019608  0.4745098  0.16470589]
  [0.32941177 0.5137255  0.19607843]
  [0.30980393 0.5294118  0.17254902]
  ...
  [0.27058825 0.49411765 0.19215687]
  [0.23921569 0.4627451  0.16078432]
  [0.20392157 0.42745098 0.1254902 ]]

 [[0.31764707 0.5176471  0.20784314]
  [0.31764707 0.52156866 0.2       ]
  [0.3137255  0.5411765  0.20784314]
  ...
  [0.36078432 0.5686275  0.3019608 ]
  [0.29803923 0.5058824  0.23921569]
  [0.26666668 0.4745098  0.20784314]]

 ...

 [[0.1254902  0.26666668 0.07843138]
  [0.14117648 0.2901961  0.09019608]
  [0.21960784 0.3882353  0.16862746]
  ...
  [0.7647059  0.92156863 0.6745098 ]
  [0.7372549  0.89411765 0.65882355]
  [0.7254902  0.88235295 0.64705884]]

 [[0.07843138 0.21960784 0.03137255]
  [0.11764706 0.26666668 0.06666667]


In [ ]:
#DATASET CLEANING!
indices= np.array([ 58, 95, 137, 138, 171, 207, 338,  412, 434, 486, 506, 529, 571, 599, 622, 658, 692, 701, 723, 725, 753, 779, 783, 827, 840, 880, 898, 901, 961, 971, 974, 989,
 1028, 1044, 1064, 1065, 1101, 1149, 1172, 1190, 1191, 1265, 1268, 1280, 1333, 1384, 1443, 1466, 1483, 1528, 1541, 1554, 1594, 1609, 1630, 1651, 1690, 1697, 1752, 1757, 1759,
 1806, 1828, 1866, 1903, 1938, 1939, 1977, 1981, 1988, 2022, 2081, 2090, 2150, 2191, 2192, 2198, 2261, 2311, 2328, 2348, 2380, 2426, 2435, 2451, 2453, 2487, 2496, 2515, 2564, 2581,
 2593, 2596, 2663, 2665, 2676, 2727, 2734, 2736, 2755, 2779, 2796, 2800, 2830, 2831, 2839, 2864, 2866, 2889, 2913, 2929, 2937, 3033, 3049, 3055, 3086, 3105, 3108, 3144, 3155, 3286,
 3376, 3410, 3436, 3451, 3488, 3490, 3572, 3583, 3666, 3688, 3700, 3740, 3770, 3800, 3801, 3802, 3806, 3811, 3821, 3835, 3862, 3885, 3896, 3899, 3904, 3927, 3931, 3946, 3950, 3964,
 3988, 3989, 4049, 4055, 4097, 4100, 4118, 4144, 4150, 4282, 4310, 4314, 4316, 4368, 4411, 4475, 4476, 4503, 4507, 4557, 4605, 4618, 4694, 4719, 4735, 4740, 4766, 4779, 4837, 4848,
 4857, 4860, 4883, 4897, 4903, 4907, 4927, 5048, 5080, 5082, 5121, 5143, 5165, 5171])
print(indices.shape)
mask = np.ones(len(images), dtype=bool)
mask[indices]=False
images = images[mask]
labels = labels[mask]
print(images.shape)

(195,)
(5005, 96, 96, 3)


#Split dataset

In [ ]:
labels = tfk.utils.to_categorical(labels,len(np.unique(labels))) #one hot encoding

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, random_state=seed, test_size=.25, stratify=labels)
print(X_train.shape[0], X_val.shape[0])

3753 1252


In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 32
epochs = 100

# Print input shape, batch size, and number of epochs
print(f"Input Shape: {input_shape}, Output Shape: {output_shape}, Batch Size: {batch_size}, Epochs: {epochs}")

#Data augmentation

In [ ]:
%pip install keras_cv

In [ ]:
import keras_cv

In [ ]:
cutmix = keras_cv.layers.CutMix(10)
output = cutmix({"images": X_train, "labels": y_train})

In [ ]:
mixup = keras_cv.layers.MixUp(10)
output_mixup = mixup({"images":X_train, "labels": y_train})

In [ ]:
output_cutmix_mixup = mixup({"images":output["images"], "labels": output["labels"]})

In [ ]:
x_train_cutmix = output["images"]
x_train_mixup = output_mixup["images"]
x_train_cutmixup = output_cutmix_mixup["images"]

y_train_cutmix = output["labels"]
y_train_mixup = output_mixup["labels"]
y_train_cutmixup = output_cutmix_mixup["labels"]

In [ ]:
X_train = np.concatenate((X_train, x_train_cutmix, x_train_mixup, x_train_cutmixup), axis = 0)

In [ ]:
y_train = np.concatenate((y_train, y_train_cutmix, y_train_mixup, y_train_cutmixup), axis = 0)

In [ ]:
del x_train_cutmix
del x_train_mixup
del x_train_cutmixup

In [ ]:
del y_train_cutmix
del y_train_mixup
del y_train_cutmixup

In [ ]:
del output
del output_mixup
del output_cutmix_mixup

In [ ]:
# The function will distribute the samples uniformly over dataset
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
X_train_augmented_shuffled, y_train_augmented_shuffled = unison_shuffled_copies(X_train, y_train)

#Transfer Learning on MobileNet

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input

In [ ]:
mobile = tfk.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg',
)
tfk.utils.plot_model(mobile, show_shapes=True)

Install Keras Tuner to do hyperparameter tuning

In [ ]:
!pip install keras-tuner -q

In [ ]:
# Use the supernet as feature extractor, i.e. freeze all its weigths
import keras_tuner
def build_model(hp):
  mobile.trainable = False
  tf.random.set_seed(seed)

  preprocessing = tf.keras.Sequential([ #a layer that (in fact is composed by multiples) applies the augmentation
      tfkl.RandomFlip("vertical"),
      tfkl.RandomFlip("horizontal"),
      tfkl.RandomRotation(0.5),
      tfkl.RandomZoom(0.3)
  ], name='preprocessing')

  # Create an input layer with shape (96, 96, 3)
  inputs = tfk.Input(shape=(96, 96, 3))
  # Connect MobileNetV2 to the input
  preprocessing = preprocessing(inputs) #perform augmentation, then connect to the same architectures
  x = mobile(preprocessing)
  #We put for both the dense layers, the parameter hp.Int representing the function Int of hp thanks to which we will do hyperparameter search through the values defined with a specific step
  x = tfkl.Dense(hp.Int('units', min_value = 128, max_value = 256, step = 64), kernel_initializer=tfk.initializers.HeUniform(seed=seed), name='HiddenDense1')(x)
  x = tfkl.Activation('relu', name='HiddenActivation1')(x)
  x = tfkl.BatchNormalization()(x)
  x = tfkl.Dense(hp.Int('units', min_value = 128, max_value = 256, step = 64), kernel_initializer=tfk.initializers.HeUniform(seed=seed), name='HiddenDense2')(x)
  x = tfkl.Activation('relu', name='HiddenActivation2')(x)
  x = tfkl.BatchNormalization()(x)

  # Add a Dense layer with 2 units and softmax activation as the classifier
  outputs = tfkl.Dense(2, activation='softmax')(x)

  # Create a Model connecting input and output
  tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

  # Compile the model with Categorical Cross-Entropy loss and Adam optimizer
  tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(1e-4,weight_decay=5e-4), metrics=['accuracy'])

  # Display model summary
  tl_model.summary()
  return tl_model

In [ ]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3
)#We create the object tuner, defining the parameter of the hyperparameter search, specifying that for each parameter value we want 3 executions, with a max of 5

In [ ]:
tuner.search(
    X_train_augmented_shuffled,
    y_train_augmented_shuffled,
    epochs = 150,
    validation_data = (X_val, y_val))#We start the search